# NFT Portfolio Analyzer

Project : The purpose of our project is to uncover the types of events that are mostly associated with a porfolio or holdings. Events in nfts's can be described as 'created','successful','cancelled','offer_entered','bid_entered' 'bid_withdrawn', 'transfer', 'approve', 'custom' and  'payout'. we want to be able to sort our holding(collections) by all these different events. It would be ideal to automate any given portfilio on opensea and have it tell us the type of events that occured the most on those holding. The project will help students decide which user on opensea is transfering, creating, approving and how often he/she is getting bids on their holdings.

In [94]:
# Initial imports
import os
import requests
import pandas as pd
import panel as pn
import pprint
import hvplot.pandas
import alpaca_trade_api as tradeapi


In [95]:
#Calling the events API with parameters
import requests

url = "https://api.opensea.io/api/v1/events?only_opensea=false&offset=0&limit=300"
headers = {"Accept": "application/json"}
params = {"asset_contract_address":"0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d",
          "collection_slug":"boredapeyachtclub",
          "token_id":"1662",
          "account_address":"0x64597fd32302d26ea7f35cacf0ef2cba56f5d7a8",
          "event_type":"transfer",
          #"auction_type":"english",
          #"occurred_before":'2021-08-11T04:05:28',
          #"occurred_after":"2021-09-22T03:03:43.731860"
          }
          

output_events = requests.request("GET", url, headers=headers, params=params )

output_events_json1 = (output_events.json())
          
pprint.pprint(output_events_json1)




{'asset_events': [{'approved_account': None,
                   'asset': {'animation_original_url': None,
                             'animation_url': None,
                             'asset_contract': {'address': '0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d',
                                                'asset_contract_type': 'non-fungible',
                                                'buyer_fee_basis_points': 0,
                                                'created_date': '2021-04-22T03:03:43.731860',
                                                'default_to_fiat': False,
                                                'description': 'The Bored Ape '
                                                               'Yacht Club is '
                                                               'a collection '
                                                               'of 10,000 '
                                                               'unique Bored '
           

In [96]:
#Calling asset dict from Events API with params
asset = [x['asset'] for x in output_events_json1['asset_events']]
collection = asset[0]['collection']
owner = asset[0]['owner']
from_account = [x['from_account'] for x in output_events_json1['asset_events']]
to_account = [x['to_account'] for x in output_events_json1['asset_events']]
from_account
to_account

[{'user': {'username': 'DKBBQs_Vault'},
  'profile_img_url': 'https://storage.googleapis.com/opensea-static/opensea-profile/18.png',
  'address': '0x64597fd32302d26ea7f35cacf0ef2cba56f5d7a8',
  'config': ''}]

In [97]:
#calling major keys From API and casting into a dataframe.
from_account = [x['from_account'] for x in output_events_json1['asset_events']]
to_account = [x['to_account'] for x in output_events_json1['asset_events']]
name =[x['asset_contract']['name']for x in asset]
created_date = [x['asset_contract']['created_date']for x in asset]
total_supply = [x['asset_contract']['total_supply']for x in asset]
token_id = [x['token_id']for x in asset]
num_sales = [x['num_sales']for x in asset]
image_url = [x['image_url']for x in asset]
symbol =[x['asset_contract']['symbol']for x in asset]
description = [x['description']for x in asset]
#print(asset)
#print(token_id)
#print(num_sales)
#print(name)
#print(image_url)
#print(asset_contract)
#print(description)


df3 = pd.DataFrame({'Name':name,'created date':created_date,'Token ID':token_id,'Total supply':total_supply,'Number of sales':num_sales,'Image_url':image_url,'symbol':symbol,'description':description,'From account':from_account,'To account':to_account})
df3

,Name,created date,Token ID,Total supply,Number of sales,Image_url,symbol,description,From account,To account
0,BoredApeYachtClub,2021-04-22T03:03:43.731860,1662,0,3,https://lh3.googleusercontent.com/UTMX1ed3wKHQ...,BAYC,None,"{'user': {'username': 'Cryptonis'}, 'profile_i...","{'user': {'username': 'DKBBQs_Vault'}, 'profil..."


In [98]:
#Calling different events that are collable with the API.

#url = "https://api.opensea.io/api/v1/events?event_type=transfer&only_opensea=false&offset=0&limit=20"
url = "https://api.opensea.io/api/v1/events?only_opensea=false&offset=0&limit=30"
#url = "https://api.opensea.io/api/v1/events?event_type=created&only_opensea=false&offset=0&limit=20"
#url = "https://api.opensea.io/api/v1/events?event_type=successful&only_opensea=false&offset=0&limit=20"
#url = "https://api.opensea.io/api/v1/events?event_type=cancelled&only_opensea=false&offset=0&limit=20"
#url = "https://api.opensea.io/api/v1/events?event_type=offer_entered&only_opensea=false&offset=0&limit=20"
#url = "https://api.opensea.io/api/v1/events?event_type=bid_entered&only_opensea=false&offset=0&limit=20"
#url = "https://api.opensea.io/api/v1/events?event_type=approve&only_opensea=false&offset=0&limit=20"
#url = "https://api.opensea.io/api/v1/events?event_type=custom&only_opensea=false&offset=0&limit=20"
#url = "https://api.opensea.io/api/v1/events?event_type=payout&only_opensea=false&offset=0&limit=20"
#url = "https://api.opensea.io/api/v1/events?asset_contract_address=0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d&collection_slug=boredapeyachtclub&token_id=1662&account_address=0x64597fd32302d26ea7f35cacf0ef2cba56f5d7a8&event_type=transfer&only_opensea=false&offset=0&limit=20"
class Events:
    def __init__ (self, url,):
        self.url=url
        self.get_data()
        self.get_assets()
        
        
    def get_data(self):
        self.data=requests.request("Get",self.url).json()

    def get_assets(self):
        self.asset=[x['asset']for x in self.data['asset_events']]
        self.token_id = [x['token_id']for x in self.asset]
        self.num_sales = [x['num_sales']for x in self.asset]
        self.image_url = [x['image_url']for x in self.asset]
        self.asset_contract = [x['asset_contract']['name']for x in self.asset]
        self.name = [x['collection']for x in self.asset]
        self.description = [x['description']for x in self.asset]
        self.from_account = [x['from_account'] for x in self.data['asset_events']]
        self.to_account = [x['to_account'] for x in self.data['asset_events']]
        self.event_type = [x['event_type'] for x in self.data['asset_events']]
        self.collection_slug = [x['collection_slug'] for x in self.data['asset_events']]
        self.starting_price = [x['starting_price'] for x in self.data['asset_events']]
        self.ending_price = [x['ending_price'] for x in self.data['asset_events']]
        
    def as_df(self): # cast attributes into df
        return pd.DataFrame({'Asset':self.asset,'Token ID':self.token_id,'Num_sales':self.num_sales,'Image':self.image_url,'asset_contract':self.asset_contract,'Description':self.description,'from_accounts':self.from_account,'to_account':self.to_account,'Event_type':self.event_type,'Collection_slug':self.collection_slug,'starting_price':self.starting_price,'Ending_price':self.ending_price})


portfolio=Events(url)
Data=portfolio.as_df()
Data.head(25)








,Asset,Token ID,Num_sales,Image,asset_contract,Description,from_accounts,to_account,Event_type,Collection_slug,starting_price,Ending_price
0,"{'id': 50759330, 'token_id': '2110', 'num_sale...",2110,2,https://lh3.googleusercontent.com/Z28fYt5VfIgg...,Borpacasso,None,"{'user': None, 'profile_img_url': 'https://sto...",None,offer_entered,borpacasso,None,None
1,"{'id': 44965284, 'token_id': '197', 'num_sales...",197,0,https://lh3.googleusercontent.com/Q2OV5JwTpuf-...,Infinites,Infinites is a collection of unique 512 artwor...,"{'user': None, 'profile_img_url': 'https://sto...",None,offer_entered,infinites-ai,None,None
2,"{'id': 44972595, 'token_id': '349', 'num_sales...",349,0,https://lh3.googleusercontent.com/1UDRlAeR_EC5...,Infinites,Infinites is a collection of unique 512 artwor...,"{'user': None, 'profile_img_url': 'https://sto...",None,offer_entered,infinites-ai,None,None
3,"{'id': 42454214, 'token_id': '17668', 'num_sal...",17668,0,https://lh3.googleusercontent.com/Yq1Xl97UfB5t...,MutantApeYachtClub,None,"{'user': {'username': 'JPEG_HOLDER'}, 'profile...",None,offer_entered,mutant-ape-yacht-club,None,None
4,"{'id': 33179191, 'token_id': '2862', 'num_sale...",2862,3,https://lh3.googleusercontent.com/yfbeBag5-6Bh...,Vogu,None,"{'user': {'username': 'vogumaster42069'}, 'pro...",None,offer_entered,vogu,None,None
5,"{'id': 20046653, 'token_id': '12975', 'num_sal...",12975,0,https://lh3.googleusercontent.com/YvWTdR3Gnk1I...,ZED Horse,None,"{'user': None, 'profile_img_url': 'https://sto...",None,bid_entered,zed-run-official,117000000000000000,117000000000000000
6,"{'id': 77819385, 'token_id': '6608511951103725...",6608511951103725465084364766382473666614669933...,0,https://lh3.googleusercontent.com/C2uee6Cb2P6m...,OpenSea Collection,A collection of NFTs with 50% of initial sales...,"{'user': {'username': 'charityrarities'}, 'pro...",None,created,birdowls,15000000000000000,15000000000000000
7,"{'id': 29241066, 'token_id': '6668', 'num_sale...",6668,6,https://lh3.googleusercontent.com/_G2fKRekBOUJ...,TheWickedCraniums,None,"{'user': {'username': 'goatmaster42069'}, 'pro...",None,offer_entered,thewickedcraniums,None,None
8,"{'id': 28742964, 'token_id': '89000612', 'num_...",89000612,0,https://lh3.googleusercontent.com/FiiNN1qceJGw...,Art Blocks,Dreams is an exploration of uneven subdivision...,"{'user': {'username': None}, 'profile_img_url'...",None,offer_entered,dreams-by-joshua-bagley,None,None
9,"{'id': 71968464, 'token_id': '4782', 'num_sale...",4782,1,https://lh3.googleusercontent.com/cSOm5LqfW8TT...,Jungle Freaks,None,"{'user': None, 'profile_img_url': 'https://sto...",None,offer_entered,jungle-freaks-by-trosley,None,None


In [99]:
#Calling Events API On one specific asset from boredApeYachtClub to retreive all the events that has occured/

url = "https://api.opensea.io/api/v1/events?asset_contract_address=0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d&collection_slug=boredapeyachtclub&token_id=1662&account_address=0x64597fd32302d26ea7f35cacf0ef2cba56f5d7a8&only_opensea=false&offset=0&limit=20"

class Events_Ape:
    def __init__ (self, url,):
        self.url=url
        self.get_data()
        self.get_assets()
        
        
    def get_data(self):
        self.data=requests.request("Get",self.url).json()

    def get_assets(self):
        self.asset=[x['asset']for x in self.data['asset_events']]
        self.token_id = [x['token_id']for x in self.asset]
        self.num_sales = [x['num_sales']for x in self.asset]
        self.image_url = [x['image_url']for x in self.asset]
        self.asset_contract = [x['asset_contract']['name']for x in self.asset]
        self.name = [x['collection']for x in self.asset]
        self.description = [x['description']for x in self.asset]
        self.from_account = [x['from_account'] for x in self.data['asset_events']]
        self.to_account = [x['to_account'] for x in self.data['asset_events']]
        self.event_type = [x['event_type'] for x in self.data['asset_events']]
        self.collection_slug = [x['collection_slug'] for x in self.data['asset_events']]
        self.starting_price = [x['starting_price'] for x in self.data['asset_events']]
        self.ending_price = [x['ending_price'] for x in self.data['asset_events']]
     
    
    def as_df(self): # cast attributes into df
        return pd.DataFrame({'Asset':self.asset,'Token ID':self.token_id,'Num_sales':self.num_sales,'Image':self.image_url,'asset_contract':self.asset_contract,'Description':self.description,'from_accounts':self.from_account,'to_account':self.to_account,'Event_type':self.event_type,'Collection_slug':self.collection_slug,'starting_price':self.starting_price,'Ending_price':self.ending_price})
portfolio1=Events_Ape(url)
Data1=portfolio1.as_df()
Data1



,Asset,Token ID,Num_sales,Image,asset_contract,Description,from_accounts,to_account,Event_type,Collection_slug,starting_price,Ending_price
0,"{'id': 23130908, 'token_id': '1662', 'num_sale...",1662,3,https://lh3.googleusercontent.com/UTMX1ed3wKHQ...,BoredApeYachtClub,None,None,None,cancelled,boredapeyachtclub,None,None
1,"{'id': 23130908, 'token_id': '1662', 'num_sale...",1662,3,https://lh3.googleusercontent.com/UTMX1ed3wKHQ...,BoredApeYachtClub,None,None,None,cancelled,boredapeyachtclub,None,None
2,"{'id': 23130908, 'token_id': '1662', 'num_sale...",1662,3,https://lh3.googleusercontent.com/UTMX1ed3wKHQ...,BoredApeYachtClub,None,None,None,cancelled,boredapeyachtclub,None,None
3,"{'id': 23130908, 'token_id': '1662', 'num_sale...",1662,3,https://lh3.googleusercontent.com/UTMX1ed3wKHQ...,BoredApeYachtClub,None,"{'user': {'username': 'DKBBQs_Vault'}, 'profil...",None,created,boredapeyachtclub,37370000000000000000,37370000000000000000
4,"{'id': 23130908, 'token_id': '1662', 'num_sale...",1662,3,https://lh3.googleusercontent.com/UTMX1ed3wKHQ...,BoredApeYachtClub,None,"{'user': {'username': 'DKBBQs_Vault'}, 'profil...",None,created,boredapeyachtclub,39990000000000000000,39990000000000000000
5,"{'id': 23130908, 'token_id': '1662', 'num_sale...",1662,3,https://lh3.googleusercontent.com/UTMX1ed3wKHQ...,BoredApeYachtClub,None,"{'user': {'username': 'DKBBQs_Vault'}, 'profil...",None,created,boredapeyachtclub,42490000000000000000,42490000000000000000
6,"{'id': 23130908, 'token_id': '1662', 'num_sale...",1662,3,https://lh3.googleusercontent.com/UTMX1ed3wKHQ...,BoredApeYachtClub,None,None,None,cancelled,boredapeyachtclub,None,None
7,"{'id': 23130908, 'token_id': '1662', 'num_sale...",1662,3,https://lh3.googleusercontent.com/UTMX1ed3wKHQ...,BoredApeYachtClub,None,"{'user': {'username': 'DKBBQs_Vault'}, 'profil...",None,created,boredapeyachtclub,43990000000000000000,43990000000000000000
8,"{'id': 23130908, 'token_id': '1662', 'num_sale...",1662,3,https://lh3.googleusercontent.com/UTMX1ed3wKHQ...,BoredApeYachtClub,None,"{'user': {'username': 'Cryptonis'}, 'profile_i...","{'user': {'username': 'DKBBQs_Vault'}, 'profil...",transfer,boredapeyachtclub,None,None


In [100]:
#calling the Collections Api with no Parameters. Generally All keys and values are empty since it does not know which portfolio it's retreiving.
url = "https://api.opensea.io/api/v1/collections?offset=0&limit=300"
collection = requests.request("GET", url, headers=headers)

collection_json = (collection.json())
collection_json

{'collections': [{'primary_asset_contracts': [],
   'traits': {},
   'stats': {'one_day_volume': 0.0,
    'one_day_change': 0.0,
    'one_day_sales': 0.0,
    'one_day_average_price': 0.0,
    'seven_day_volume': 0.0,
    'seven_day_change': 0.0,
    'seven_day_sales': 0.0,
    'seven_day_average_price': 0.0,
    'thirty_day_volume': 0.0,
    'thirty_day_change': 0.0,
    'thirty_day_sales': 0.0,
    'thirty_day_average_price': 0.0,
    'total_volume': 0.0,
    'total_sales': 0.0,
    'total_supply': 0.0,
    'count': 0.0,
    'num_owners': 0,
    'average_price': 0.0,
    'num_reports': 0,
    'market_cap': 0.0,
    'floor_price': 0.0},
   'banner_image_url': None,
   'chat_url': None,
   'created_date': '2021-10-26T21:06:18.186068',
   'default_to_fiat': False,
   'description': None,
   'dev_buyer_fee_basis_points': '0',
   'dev_seller_fee_basis_points': '0',
   'discord_url': None,
   'display_data': {'card_display_style': 'contain', 'images': []},
   'external_url': None,
   'feat

In [101]:
#Retreiving Collection API with for a specfic person that owns that collection
url = "https://api.opensea.io/api/v1/collections?offset=0&limit=300"
params = {"asset_owner":"0x64597fd32302d26ea7f35cacf0ef2cba56f5d7a8"}
collection = requests.request("GET", url, headers=headers, params=params)

collection_json1 = (collection.json())
collection_json1

[{'primary_asset_contracts': [{'address': '0x4e962d488412a14aa37eacadcb83f18c7e2271a7',
    'asset_contract_type': 'non-fungible',
    'created_date': '2021-10-18T00:05:05.840735',
    'name': 'Robopets',
    'nft_version': '3.0',
    'opensea_version': None,
    'owner': 95068269,
    'schema_name': 'ERC721',
    'symbol': 'ROBOPET',
    'total_supply': '0',
    'description': "Robopets is a collection of algorithmically generated pets meant to be the perfect companions to Robotos! \r\n\r\nSome are doggos who love to cuddle, others are playful kitties that want to run around with their yacht, and some may just want to chew your leg off!\r\n\r\nEvery Roboto will be able to adopt a Robopet for free, you'll only have to pay for gas.\r\n\r\n**Other Official Robotos Collections**\r\n\r\n[Robotos](https://opensea.io/collection/robotos-official)\r\n\r\n[Robotos Originals](https://opensea.io/collection/robotos-originals)\r\n\r\n[Emojibotos](https://opensea.io/collection/emojibotos)\r\n\r\n[Ro

In [102]:
#Appending several keys from collections API to retreive all the neccessery info for each specifc item that a  person is holding in his collection or portfolio
names=[]
for name in collection_json1:
    for x in name['primary_asset_contracts']:
        names.append(x['name'])
print(names)

nft_versions=[]
for nft_version in collection_json1:
    for x in nft_version['primary_asset_contracts']:
        nft_versions.append(x['nft_version'])
print(nft_versions)


symbols=[]
for symbol in collection_json1:
    for x in symbol['primary_asset_contracts']:
        symbols.append(x['symbol'])
print(symbols)


descriptions=[]
for description in collection_json1:
    for x in description['primary_asset_contracts']:
        descriptions.append(x['description'])
print(descriptions)


image_urls=[]
for image_url in collection_json1:
    for x in image_url['primary_asset_contracts']:
        image_urls.append(x['image_url'])
print(image_urls)


payout_adresses=[]
for payout_address in collection_json1:
    for x in payout_address['primary_asset_contracts']:
        payout_adresses.append(x['payout_address'])
print(payout_adresses)

#The following keys could not be Appended, so they were called individually.

['Robopets', 'KaijuKingz', 'FoxFam', 'FRYCOOKS', 'The Blitnauts', 'cLoot.org', 'BadCache', 'CyberKongz VX', 'DeadFellaz', 'Robotos', 'Cool Cats', 'J48BA RECIPES', 'J48BATRINKETS', 'Blockbob', 'NiftyPins', 'Blitmap', 'Pulsquares', 'BoredApeYachtClub', 'J48BAFORMS', 'parallel', 'BASTARD GAN PUNKS V2', 'FND NFT', 'The Sandbox Official Open Sea Store', "Sandbox's ASSETs", "Sandbox's ASSETs", "Sandbox's LANDs", 'SuperRare', 'Unidentified contract', 'SuperRare', 'CryptoPunks']
['3.0', '3.0', '3.0', '3.0', '3.0', '3.0', '3.0', '3.0', '3.0', '3.0', '3.0', '3.0', '3.0', None, None, '3.0', '3.0', '3.0', '3.0', None, '3.0', '3.0', '3.0', None, 'unsupported', '3.0', 'unsupported', None, '3.0', 'unsupported']
['ROBOPET', 'KAIJU', 'FOXFAM', 'FRYCOOK', 'NAUT', 'CLOOT', 'BadCache', 'KONGZ VX', 'DEADFELLAZ', 'ROBO', 'COOL', 'RECIPE', 'TRINKET', 'BLOB', 'NIFTYPINS', 'BLIT', 'PULS', 'BAYC', 'JABBA', 'LL', 'BGANPUNKV2', 'FNDNFT', 'SAND', 'ASSET', '', '', 'SUPR', '', '', 'PUNK']
["Robopets is a collection 

In [103]:
#Casting those keys and its values into a DataFrame and all the different collections that person owns in his portfolio
df=pd.DataFrame({'Name':names,'NFT Version':nft_versions,'symbol':symbols,'Descriptions':descriptions,'Image_urls':image_urls,'payout_address':payout_adresses})
df

,Name,NFT Version,symbol,Descriptions,Image_urls,payout_address
0,Robopets,3.0,ROBOPET,Robopets is a collection of algorithmically ge...,https://lh3.googleusercontent.com/fy07Rgzu6zum...,0x273dc0347cb3aba026f8a4704b1e1a81a3647cf3
1,KaijuKingz,3.0,KAIJU,"3,333 Genesis Kaiju Kingz created by Augminted...",https://lh3.googleusercontent.com/bQvXzE_Pn8Sw...,0x76b704fd0d9bb603f3c1afcff9d022fb224893c0
2,FoxFam,3.0,FOXFAM,Fox Fam is a collection of generative characte...,https://lh3.googleusercontent.com/ZdUz4EOKP8It...,0x6e1c38bb640463558d29f0d8fecc4d1f6fe960a9
3,FRYCOOKS,3.0,FRYCOOK,SUDOBURGER presents FRYCOOKs - your entry poin...,https://lh3.googleusercontent.com/JlJfwmU5VPPQ...,0x01c76cf5677bd9e4b4e2acd06cd3b0c7a39f94cc
4,The Blitnauts,3.0,NAUT,The Blitnauts is the first expansion pack for ...,https://lh3.googleusercontent.com/0tU9duZZsSeV...,None
5,cLoot.org,3.0,CLOOT,cLoot is randomized adventurer gear generated ...,https://lh3.googleusercontent.com/4Obv8mnsVvQB...,None
6,BadCache,3.0,BadCache,"This is an original, programmable and generati...",https://lh3.googleusercontent.com/q6j-T5GykT4Q...,0xbd40add28615febf5bda18afea24c7908695210b
7,CyberKongz VX,3.0,KONGZ VX,CyberKongz VX are unique and randomly generate...,https://lh3.googleusercontent.com/vpJFZRfBl9gh...,0xeef61546a3e2ffe36cf7d9c826c76edcb0dc6e22
8,DeadFellaz,3.0,DEADFELLAZ,"10,000 undead NFTs on the Ethereum blockchain....",https://lh3.googleusercontent.com/seJEwLWJP3RA...,0xe9d30eddd11dea8433cf6d2b2c22e9cce94113dc
9,Robotos,3.0,ROBO,Robotos is a collection of algorithmically gen...,https://lh3.googleusercontent.com/5PO4I0APxP3x...,0x4977824d1484ef684f51607abcab831c160158af


In [104]:
#The following keys could not be Appended, so they were called individually. Those were all the important stats from owning each specific NFTs.
stats=[x['stats']for x in collection_json1]
one_day_sales=[x['one_day_sales']for x in stats]
seven_day_sales=[x['seven_day_sales']for x in stats]
thirty_day_sales=[x['thirty_day_sales']for x in stats]
total_supply=[x['total_supply']for x in stats]
num_owners=[x['num_owners']for x in stats]
average_price=[x['average_price']for x in stats]
market_cap=[x['market_cap']for x in stats]
floor_price=[x['floor_price']for x in stats]
one_day_volume=[x['one_day_volume']for x in stats]
floor_price

df2=pd.DataFrame({'one_day_sales':one_day_sales,'one_day_volume':one_day_volume,'seven_day_sales':seven_day_sales,'thirty_day_sales':thirty_day_sales,'total_supply':total_supply,'num_owners':num_owners,'average_price':average_price,'market_cap':market_cap,'floor_price':floor_price})
df2                                  

,one_day_sales,one_day_volume,seven_day_sales,thirty_day_sales,total_supply,num_owners,average_price,market_cap,floor_price
0,48.0,6.905500,229.0,317.0,8017.0,3858,0.129575,1.032513e+03,0.147900
1,32.0,158.573000,555.0,2578.0,3394.0,1766,4.635207,1.808976e+04,2.990000
2,0.0,0.000000,0.0,0.0,1.0,38,0.000000,0.000000e+00,0.050000
3,47.0,13.097800,670.0,10484.0,10000.0,3035,0.306170,3.035869e+03,0.220000
4,3.0,0.550000,5.0,36.0,1899.0,1001,0.104093,2.183850e+02,0.009000
5,0.0,0.000000,3.0,26.0,756.0,396,3.818845,2.646000e+03,4.000000
6,0.0,0.000000,0.0,0.0,2454.0,1215,0.130270,0.000000e+00,0.003500
7,0.0,0.000000,0.0,3.0,30.0,26,0.288500,0.000000e+00,0.200000
8,0.0,0.000000,0.0,0.0,84.0,27,0.693667,0.000000e+00,0.490000
9,24.0,54.802900,332.0,2369.0,13965.0,3562,1.240121,2.171969e+04,1.240000


In [105]:
#Combining Two DataFrames from the same APIs since they could not be created together.
combined_df= pd.concat([df,df2], axis="columns" , join="inner")
combined_df.head(30)

,Name,NFT Version,symbol,Descriptions,Image_urls,payout_address,one_day_sales,one_day_volume,seven_day_sales,thirty_day_sales,total_supply,num_owners,average_price,market_cap,floor_price
0,Robopets,3.0,ROBOPET,Robopets is a collection of algorithmically ge...,https://lh3.googleusercontent.com/fy07Rgzu6zum...,0x273dc0347cb3aba026f8a4704b1e1a81a3647cf3,48.0,6.905500,229.0,317.0,8017.0,3858,0.129575,1032.512581,0.1479
1,KaijuKingz,3.0,KAIJU,"3,333 Genesis Kaiju Kingz created by Augminted...",https://lh3.googleusercontent.com/bQvXzE_Pn8Sw...,0x76b704fd0d9bb603f3c1afcff9d022fb224893c0,32.0,158.573000,555.0,2578.0,3394.0,1766,4.635207,18089.758399,2.9900
2,FoxFam,3.0,FOXFAM,Fox Fam is a collection of generative characte...,https://lh3.googleusercontent.com/ZdUz4EOKP8It...,0x6e1c38bb640463558d29f0d8fecc4d1f6fe960a9,0.0,0.000000,0.0,0.0,1.0,38,0.000000,0.000000,0.0500
3,FRYCOOKS,3.0,FRYCOOK,SUDOBURGER presents FRYCOOKs - your entry poin...,https://lh3.googleusercontent.com/JlJfwmU5VPPQ...,0x01c76cf5677bd9e4b4e2acd06cd3b0c7a39f94cc,47.0,13.097800,670.0,10484.0,10000.0,3035,0.306170,3035.868675,0.2200
4,The Blitnauts,3.0,NAUT,The Blitnauts is the first expansion pack for ...,https://lh3.googleusercontent.com/0tU9duZZsSeV...,None,3.0,0.550000,5.0,36.0,1899.0,1001,0.104093,218.385000,0.0090
5,cLoot.org,3.0,CLOOT,cLoot is randomized adventurer gear generated ...,https://lh3.googleusercontent.com/4Obv8mnsVvQB...,None,0.0,0.000000,3.0,26.0,756.0,396,3.818845,2646.000000,4.0000
6,BadCache,3.0,BadCache,"This is an original, programmable and generati...",https://lh3.googleusercontent.com/q6j-T5GykT4Q...,0xbd40add28615febf5bda18afea24c7908695210b,0.0,0.000000,0.0,0.0,2454.0,1215,0.130270,0.000000,0.0035
7,CyberKongz VX,3.0,KONGZ VX,CyberKongz VX are unique and randomly generate...,https://lh3.googleusercontent.com/vpJFZRfBl9gh...,0xeef61546a3e2ffe36cf7d9c826c76edcb0dc6e22,0.0,0.000000,0.0,3.0,30.0,26,0.288500,0.000000,0.2000
8,DeadFellaz,3.0,DEADFELLAZ,"10,000 undead NFTs on the Ethereum blockchain....",https://lh3.googleusercontent.com/seJEwLWJP3RA...,0xe9d30eddd11dea8433cf6d2b2c22e9cce94113dc,0.0,0.000000,0.0,0.0,84.0,27,0.693667,0.000000,0.4900
9,Robotos,3.0,ROBO,Robotos is a collection of algorithmically gen...,https://lh3.googleusercontent.com/5PO4I0APxP3x...,0x4977824d1484ef684f51607abcab831c160158af,24.0,54.802900,332.0,2369.0,13965.0,3562,1.240121,21719.689798,1.2400
